In [76]:
import json
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv
import os
from openai import OpenAI
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY")) # Get Key from Env (ignored in git) 
client = OpenAI()
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')
from tqdm.auto import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [60]:
# Path to users.json
users_path = Path("../context/users/users.json")

In [61]:
def load_users():
    if users_path.exists():
        with open(users_path, "r") as f:
            return json.load(f)
    return []

In [62]:
def get_user_by_id(user_id, users):
    for user in users:
        if user.get("user_id") == user_id:
            return user
    return None

In [63]:
#get user id
user_id = input("Enter your user_id: ").strip()

In [64]:
# get user info
users = load_users() 
user = get_user_by_id(user_id, users)
if user: 
    print(f"User found: {user}")
else:
    print(f"No user found with user_id: {user_id}")
    exit(1)

User found: {'user_id': '00006', 'user_name': 'SarahWilson', 'first_name': 'Sarah', 'last_name': 'Wilson', 'created_at': '2025-06-11T20:53:48.928579', 'persona': 'steady_believer', 'view': 'christian', 'step': None, 'step_in_progress': False, 'first_sober_date': '2025-06-01'}


In [65]:
# Load spiritual profile
spiritual_profile_path = Path("../context/spiritual_profile/spiritual_profile.json")
if spiritual_profile_path.exists():
    with open(spiritual_profile_path, "r") as f:
        profiles = json.load(f)
    user_profile = next((p for p in profiles if p["user_id"] == user_id), None)
else:
    user_profile = None

In [66]:
from datetime import datetime
# store user info to variables. 
user_id = user['user_id']
first_name = user.get('first_name', 'Unknown')
last_name = user.get('last_name', 'Unknown')
view = user.get('view', 'Unknown')
persona = user.get('persona', 'Unknown')
step = user.get('step', 'Unknown')
step_in_progress = user.get('step_in_progress', 'Unknown')
first_sober_date = user.get("first_sober_date")
if first_sober_date:
    sober_days = (datetime.now() - datetime.strptime(first_sober_date, "%Y-%m-%d")).days
else:
    sober_days = None

# store spiritual profile info to variables.
openness_level = user_profile['current_profile']['openness_level']
higher_power_defined = user_profile['current_profile']['higher_power_defined']


In [67]:
print(user_id)
print(first_name)
print(last_name)
print(view)
print(persona)
print(step)
print(step_in_progress)
print(sober_days)
print(openness_level)
print(higher_power_defined)

00006
Sarah
Wilson
christian
steady_believer
None
False
11
medium
True


In [69]:
# --- Load and summarize previous conversations ---

conversations_path = Path("../context/conversations/conversations.json")
recent_convos = []
recent_cravings_convos = []

if conversations_path.exists():
    with open(conversations_path, "r") as f:
        conversations = json.load(f)
    # Filter for this user
    user_convos = [c for c in conversations if c.get("user_id") == user_id]
    # Sort by timestamp (descending, most recent first)
    user_convos = sorted(user_convos, key=lambda x: x.get("timestamp", ""), reverse=True)
    # Last 3 overall conversations
    recent_convos = user_convos[:3]
    # Last 3 cravings conversations (if any)
    cravings_convos = [c for c in user_convos if c.get("conversation_type") == "cravings"]
    recent_cravings_convos = cravings_convos[:3]
else:
    print("No conversations found.")

def summarize_convos(convos):
    summaries = []
    for c in convos:
        summary = {
            "summary": c.get("conversation_data", {}).get("summary"),
            "key_insights": c.get("key_insights"),
            "action_items": c.get("action_items"),
            "last_dialogue": c.get("conversation_data", {}).get("dialogue", []),
            "emotional_tone_end": c.get("emotional_tone_end"),
            "conversation_type": c.get("conversation_type"),
            "timestamp": c.get("timestamp")
        }
        summaries.append(summary)
    return summaries

# Summaries for last 3 overall conversations
recent_convo_summaries = summarize_convos(recent_convos)
# Summaries for last 3 cravings conversations
recent_cravings_summaries = summarize_convos(recent_cravings_convos)

# Example: print summaries for review
print("Last 3 conversations:")
for i, convo in enumerate(recent_convo_summaries, 1):
    print(f"\nConversation {i}:")
    print(f"  Summary: {convo['summary']}")
    print(f"  Key Insights: {convo['key_insights']}")
    print(f"  Action Items: {convo['action_items']}")
    print(f"  Last Dialogue: {convo['last_dialogue']}")
    print(f"  Emotional Tone End: {convo['emotional_tone_end']}")
    print(f"  Conversation Type: {convo['conversation_type']}")
    print(f"  Timestamp: {convo['timestamp']}")

print("\nLast 3 cravings conversations:")
for i, convo in enumerate(recent_cravings_summaries, 1):
    print(f"\nCravings Conversation {i}:")
    print(f"  Summary: {convo['summary']}")
    print(f"  Key Insights: {convo['key_insights']}")
    print(f"  Action Items: {convo['action_items']}")
    print(f"  Last Dialogue: {convo['last_dialogue']}")
    print(f"  Emotional Tone End: {convo['emotional_tone_end']}")
    print(f"  Conversation Type: {convo['conversation_type']}")
    print(f"  Timestamp: {convo['timestamp']}")

Last 3 conversations:

Conversation 1:
  Summary: Sarah is exploring her relationship with marijuana and is open to discussing spirituality as part of his recovery journey. He feels stuck in achieving his life goals and is curious about how others manage similar situations.
  Key Insights: ['Sarah feels marijuana is making him lazy and unproductive.', 'He wants to buy a house and have a family but feels stuck.', 'Sarah is uncertain about being part of a community but is open to exploring spirituality.']
  Action Items: ['Explore how faith might support recovery.']
  Last Dialogue: [{'role': 'user', 'content': 'i just wanted to check things out'}, {'role': 'sponsor', 'content': "I'm really glad you decided to check things out today. It's a good first step, and everyone here has taken it at some point. What's your name, if you don’t mind sharing?"}, {'role': 'user', 'content': 'Sarah'}, {'role': 'sponsor', 'content': "Hi Sarah, it's really nice to meet you. I'm glad you're here. What bro

In [71]:
recent_convo_summaries

[{'summary': 'Sarah is exploring her relationship with marijuana and is open to discussing spirituality as part of his recovery journey. He feels stuck in achieving his life goals and is curious about how others manage similar situations.',
  'key_insights': ['Sarah feels marijuana is making him lazy and unproductive.',
   'He wants to buy a house and have a family but feels stuck.',
   'Sarah is uncertain about being part of a community but is open to exploring spirituality.'],
  'action_items': ['Explore how faith might support recovery.'],
  'last_dialogue': [{'role': 'user',
    'content': 'i just wanted to check things out'},
   {'role': 'sponsor',
    'content': "I'm really glad you decided to check things out today. It's a good first step, and everyone here has taken it at some point. What's your name, if you don’t mind sharing?"},
   {'role': 'user', 'content': 'Sarah'},
   {'role': 'sponsor',
    'content': "Hi Sarah, it's really nice to meet you. I'm glad you're here. What br

In [72]:
# Format summaries and key insights for the prompt
summaries_text = "\n".join(
    [f"- {c['summary']}" for c in recent_convo_summaries if c['summary']]
)
key_insights_text = "\n".join(
    [f"- {insight}" for c in recent_convo_summaries for insight in (c['key_insights'] or [])]
)

In [73]:
summaries_text

'- Sarah is exploring her relationship with marijuana and is open to discussing spirituality as part of his recovery journey. He feels stuck in achieving his life goals and is curious about how others manage similar situations.'

In [75]:
key_insights_text

'- Sarah feels marijuana is making him lazy and unproductive.\n- He wants to buy a house and have a family but feels stuck.\n- Sarah is uncertain about being part of a community but is open to exploring spirituality.'